In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1579211849443_0005,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [2]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Load data from S3

In [3]:
data_path = "s3://msds694-group15/slow.csv"
data = ss.read.csv(data_path, inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data preprocessing

In [4]:
processed_rdd = data.rdd\
                    .map(lambda x: list(x[0:11])
                         + [min(x[11:]), max(x[11:]), float(np.mean(x[11:])), float(np.median(x[11:]))]
                         + [x[-1]])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
schema = StructType([StructField("num_prev_so", IntegerType(), True),
                     StructField("mean_amp_prev_so", DoubleType(), True),
                     StructField("mean_duration_prev_so", DoubleType(), True),
                     StructField("amp_curr_so", DoubleType(), True),
                     StructField("duration_curr_so", IntegerType(), True),
                     StructField("curr_sleep_stage", IntegerType(), True),
                     StructField("time_elapsed_since_asleep", IntegerType(), True),
                     StructField("time_deep_sleep", IntegerType(), True),
                     StructField("time_light_sleep", IntegerType(), True),
                     StructField("time_rem_sleep", IntegerType(), True),
                     StructField("time_wake_sleep", IntegerType(), True),
                     StructField("eeg_min", DoubleType(), True),
                     StructField("eeg_max", DoubleType(), True),
                     StructField("eeg_mean", DoubleType(), True),
                     StructField("eeg_median", DoubleType(), True),
                     StructField("label", IntegerType(), True)
                    ])
data_preprocessed = ss.createDataFrame(processed_rdd, schema)
data_preprocessed.printSchema()
data_preprocessed.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- num_prev_so: integer (nullable = true)
 |-- mean_amp_prev_so: double (nullable = true)
 |-- mean_duration_prev_so: double (nullable = true)
 |-- amp_curr_so: double (nullable = true)
 |-- duration_curr_so: integer (nullable = true)
 |-- curr_sleep_stage: integer (nullable = true)
 |-- time_elapsed_since_asleep: integer (nullable = true)
 |-- time_deep_sleep: integer (nullable = true)
 |-- time_light_sleep: integer (nullable = true)
 |-- time_rem_sleep: integer (nullable = true)
 |-- time_wake_sleep: integer (nullable = true)
 |-- eeg_min: double (nullable = true)
 |-- eeg_max: double (nullable = true)
 |-- eeg_mean: double (nullable = true)
 |-- eeg_median: double (nullable = true)
 |-- label: integer (nullable = true)

+-----------+------------------+---------------------+-----------+----------------+----------------+-------------------------+---------------+----------------+--------------+---------------+--------+-------+--------------------+-----------------+-----+
|num_pr

## Write processed data to HDFS

In [18]:
data_preprocessed.write.option("path", "/data").saveAsTable('data8')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read data from HDFS

In [19]:
import time
start = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
data_df = ss.sql("select * from parquet.`/data`")
data_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+---------------------+------------------+----------------+----------------+-------------------------+---------------+----------------+--------------+---------------+-------------------+-------+-------------------+------------------+-----+
|num_prev_so|  mean_amp_prev_so|mean_duration_prev_so|       amp_curr_so|duration_curr_so|curr_sleep_stage|time_elapsed_since_asleep|time_deep_sleep|time_light_sleep|time_rem_sleep|time_wake_sleep|            eeg_min|eeg_max|           eeg_mean|        eeg_median|label|
+-----------+------------------+---------------------+------------------+----------------+----------------+-------------------------+---------------+----------------+--------------+---------------+-------------------+-------+-------------------+------------------+-----+
|         87|           149.343|              266.471|           204.406|             252|               3|                     2540|            420|             600|             0|      

## Train the decision tree model

In [21]:
va = VectorAssembler(outputCol="features", inputCols=data_df.columns[0:-1])
data_vec = va.transform(data_df).select("features", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
data_vec.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[87.0,149.343,266...|    0|
|[629.0,153.59,275...|    1|
|[608.0,153.353999...|    2|
|[737.0,156.01,276...|    0|
|[115.0,148.907,26...|    2|
+--------------------+-----+
only showing top 5 rows

In [23]:
train_vec, valid_vec = data_vec.randomSplit([0.85, 0.15], seed=1)
train_vec.cache()
valid_vec.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [24]:
dt = DecisionTreeClassifier(seed=1)
dtmodel = dt.fit(train_vec)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
dtpredicts = dtmodel.transform(valid_vec)
metrics = MulticlassClassificationEvaluator()\
                .setLabelCol("label")\
                .setPredictionCol("prediction")
metrics.setMetricName("accuracy")
metrics.evaluate(dtpredicts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.494269851780166

## Cross validation

In [26]:
evaluator = MulticlassClassificationEvaluator()
evaluator.setMetricName("accuracy")
paramGrid = ParamGridBuilder().addGrid(dt.maxDepth, [5, 10, 15, 20, 25, 30]).build()
cv = CrossValidator(estimator=dt, evaluator=evaluator, numFolds=5, estimatorParamMaps=paramGrid)
cvmodel = cv.fit(train_vec)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Evaluate the best model

In [27]:
final_model = cvmodel.bestModel
max_depth = final_model._java_obj.getMaxDepth()
prediction = final_model.transform(valid_vec)
evaluator.setMetricName("accuracy")
accuracy = evaluator.evaluate(prediction)
evaluator.setMetricName("f1")
f1 = evaluator.evaluate(prediction)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Print results

In [28]:
print(f"Best max depth: {max_depth}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 score: {f1:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best max depth: 5
Accuracy: 0.4943
F1 score: 0.4657

In [29]:
end = time.time()
end - start

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

299.1189522743225